In [1]:
import random
import time
import sys
import os
import json
import datetime
import pickle
import warnings
from collections import Counter

import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置config
class TrainingConfig(object):
    epoches = 15
    evaluate_every = 100
    checkpoint_every = 100
    learning_rate = 1e-3


class ModelConfig(object):
    embedding_size = 100
    
    hidden_sizes = [128]  # LSTM结构的神经元个数，量实现多层可以在这个列表中添加hiddenSize，如[256, 128]
    
    layers = [128]  # 全连接层的hidden_size，支持多层全连接层和只有softmax层，若只有softmax层，则列表为空
    
    dropout_keep_prob = 0.5
    l2_reg_lambda = 0.0


class Config(object):
    batch_size = 64
    
    train_data_source = "../data/train.txt"
    eval_data_source = "../data/dev.txt"
    test_data_source = "../data/test.txt"
    
    num_classes = 7
    
    training = TrainingConfig()
    
    model = ModelConfig()
    
    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class DataGenerator(object):
    def __init__(self, config):
        self._train_data_source = config.train_data_source
        self._eval_data_source = config.eval_data_source
        
        self._embedding_size = config.model.embedding_size
        self._batch_size = config.batch_size
        self._num_classes = config.num_classes
        
        self.train_datas = []
        self.train_labels = []
        
        self.eval_datas = []
        self.eval_labels = []
        
        self.word_embedding =None
        
        self.vocab_len = 0 
        self._word_to_index = {}
        self.label_to_index = {}
        
    def _read_file(self, file_path):
        # 读取原始数据
        sentences = []
        labels = []
        with open(file_path, "r", encoding="utf-8") as f:
            sentence = []
            label = []
            for line in f.readlines():
                if line != "\n":
                    char, target = line.strip().split()
                    sentence.append(char)
                    label.append(target)
                else:
                    sentences.append(sentence)
                    labels.append(label)
                    sentence = []
                    label = []
        
        return sentences, labels
    
    
    def _read_vocab(self, sentences, labels):
        # 创建的词汇表
        all_words = [word for sentence in sentences for word in sentence]
        word_count = Counter(all_words)
        sort_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = ["<PAD>", "<UNK>"] + [item[0] for item in sort_word_count if item[1] > 0] 
        self.vocab_len = len(words)
        self._word_to_index = dict(zip(words, range(len(words))))
        
        with open("../data/json/word_to_index.json", "w") as f:
            json.dump(self._word_to_index, f)
        
        all_lables = list(set([item for label in labels for item in label]))
        self.label_to_index = dict(zip(all_lables, range(len(all_lables))))
        
        with open("../data/json/label_to_index.json", "w") as f:
            json.dump(self.label_to_index, f)
        
    def _process_data(self, x, is_data=True):
        # 将词或者标签转换成数字
        if is_data:
            newX = [self._word_to_index[item] if item in self._word_to_index else self._word_to_index["<UNK>"] for item in x]
            
        if not is_data:
            newX = [self.label_to_index[item] for item in x]
        
        return newX
        
    def _gen_train_eval(self, datas, labels):
        # 生成训练集和验证集
        new_datas = [self._process_data(data, is_data=True) for data in datas]
        new_labels = [self._process_data(label, is_data=False) for label in labels]
        
        return new_datas, new_labels
        
        
    def gen_data(self):
        # 调用内部方法，生成训练集和测试集
        
        # 读取txt文件
        train_datas, train_labels = self._read_file(self._train_data_source)
        eval_datas, eval_labels = self._read_file(self._eval_data_source)
        
        # 利用训练集生成映射表
        self._read_vocab(train_datas, train_labels)
        
        # 生成训练集和测试集
        train_datas, train_labels = self._gen_train_eval(train_datas, train_labels)
        eval_datas, eval_labels = self._gen_train_eval(eval_datas, eval_labels)
        
        self.train_datas = train_datas
        self.train_labels = train_labels
        self.eval_datas = eval_datas
        self.eval_labels = eval_labels
        
    def _one_hot(self, y):
        """
        对每个位置的标签用one-hot的形式表示
        """
        label_vec = [0] * self._num_classes
        new_y = []
        for labels in y:
            sub_y = []
            for label in labels:
                label_vec = [0] * self._num_classes
                label_vec[label] = 1
                sub_y.append(label_vec)
                
            new_y.append(sub_y)
        
        return new_y
                

    def _format_data(self, x, y):
        """
        对输入的数据按照最大的长度补齐
        """
        sequence_len = [len(data) for data in x]
        max_len = max(sequence_len)
        new_x = [x[i] + [self._word_to_index["<PAD>"]] * (max_len - sequence_len[i]) for i in range(len(x))]
        new_y = self._one_hot([y[i] + [self.label_to_index["O"]] * (max_len - sequence_len[i]) for i in range(len(y))])
        
        return dict(input_x=new_x, input_y = new_y, max_len=max_len, sequence_len=sequence_len)
   
    def next_batch(self, x, y):
        """
        生成batch数据集，用生成器的方式输出
        """
        # 每一个epoch时，都要打乱数据集
        mid_val = list(zip(x, y))
        random.shuffle(mid_val)
        x, y = zip(*mid_val)
        x = list(x)
        y = list(y)

        num_batches = len(x) // self._batch_size

        for i in range(num_batches):
            start = i * self._batch_size
            end = start + self._batch_size
            batch_x =x[start: end]
            batch_y = y[start: end]

            yield self._format_data(batch_x, batch_y)    
    

dataGen = DataGenerator(config)
dataGen.gen_data()

In [4]:
print("train data length: {}".format(len(dataGen.train_datas)))
print("eval data length: {}".format(len(dataGen.eval_datas)))
print("vocab length: {}".format(dataGen.vocab_len))
print("label numbers: {}".format(len(dataGen.label_to_index.keys())))

train data length: 20864
eval data length: 2318
vocab length: 4314
label numbers: 7


In [5]:
# 构建模型
class BiLSTMCrf(object):
    """
    Bi-LSTM+crf用于命名实体识别
    """
    def __init__(self, config, vocab_len, use_crf=False):

        # 定义模型的输入
        self.input_x = tf.placeholder(tf.int32, [None, None], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, None, config.num_classes], name="input_y")
        self.sequence_len = tf.placeholder(tf.int32, [None], name="sequence_len")
        self.max_len = tf.placeholder(tf.int32, name="max_len")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        self.config = config
        
        # 定义l2损失
        l2_loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 词向量初始化词嵌入矩阵
            self.W = tf.Variable(
                tf.random_uniform([vocab_len, config.model.embedding_size], -1.0, 1.0),
                name="W")
            
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embedded_words = tf.nn.embedding_lookup(self.W, self.input_x)
            
        with tf.name_scope("Bi-LSTM"):

            for idx, hidden_size in enumerate(config.model.hidden_sizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstm_fw_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True),
                                                                 output_keep_prob=self.dropout_keep_prob)
                    # 定义反向LSTM结构
                    lstm_bw_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True),
                                                                 output_keep_prob=self.dropout_keep_prob)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs, current_state = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, 
                                                                             self.embedded_words, dtype=tf.float32,
                                                                             scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2]
                    self.embedded_words = tf.concat(outputs, 2)
        
        output_size = config.model.hidden_sizes[-1] * 2  # 因为是双向LSTM，最终的输出值是fw和bw的拼接，因此要乘以2
        output = tf.reshape(self.embedded_words, [-1, output_size])  # reshape成全连接层的输入维度
        
        with tf.name_scope("hidden_layer"):
            for idx, hidden_size in enumerate(config.model.layers):
                with tf.name_scope("hidden_layer" + str(idx)):
                    hidden_w = tf.get_variable("hidden_w",
                                              shape=[output_size, hidden_size],
                                              initializer=tf.contrib.layers.xavier_initializer())
                    hidden_b = tf.Variable(tf.constant(0.1, shape=[hidden_size]), name="hidden_b")
                    l2_loss += tf.nn.l2_loss(hidden_w)
                    l2_loss += tf.nn.l2_loss(hidden_b)
                    # 加一层tanh激活函数
                    output = tf.tanh(tf.nn.xw_plus_b(output, hidden_w, hidden_b, name="hidden_output"))
                    output_size = hidden_size
        
        # 全连接层的输出
        with tf.name_scope("output_layer"):
            output_w = tf.get_variable(
                "output_w",
                shape=[output_size, config.num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            
            output_b= tf.Variable(tf.constant(0.1, shape=[config.num_classes]), name="output_b")
            l2_loss += tf.nn.l2_loss(output_w)
            l2_loss += tf.nn.l2_loss(output_b)
            self.logits = tf.nn.xw_plus_b(output, output_w, output_b, name="logits")
            self.new_logits = tf.reshape(self.logits, [-1, self.max_len, config.num_classes])
            
            # 根据传入的序列长度取出实际的序列，mask的维度[batchSize, maxLen]，mask中的值为布尔值
            mask = tf.sequence_mask(self.sequence_len)
            self.true_y = tf.argmax(self.input_y, -1, name="true_y", output_type=tf.int32)
            
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            if use_crf:
                log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(self.new_logits, self.true_y, self.sequence_len)
                self.trans_params = trans_params  # 转移矩阵
                self.loss = tf.reduce_mean(-log_likelihood)
                
            else:
                losses = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.new_logits, labels=self.input_y)
                # 根据布尔值来去除位置为True的值
                losses = tf.boolean_mask(losses, mask)
                self.loss = tf.reduce_mean(losses)
        
        with tf.name_scope("masked_output"):
            if use_crf:
                viterbi_sequence, viterbi_score = tf.contrib.crf.crf_decode(self.new_logits, self.trans_params, self.sequence_len)
                self.pred_y = viterbi_sequence
            else:
                self.predictions = tf.argmax(self.logits, 1, name="predictions", output_type=tf.int32)
                self.pred_y = tf.reshape(self.predictions, [-1, self.max_len], name="pred_y")
            
            # boolean_mask会将二维的tensor转换成一维，因为mask也是二维。
            # 维度计算公式=self.trueY的维度 - mask的维度 + 1
            self.true_y = tf.boolean_mask(self.true_y, mask, name="masked_true_y")
            self.pred_y = tf.boolean_mask(self.pred_y, mask, name="masked_pred_y")
            
        with tf.name_scope("accuracy"):
            # 计算accuracy
            self.accuracy = tf.reduce_mean(tf.cast(tf.equal(self.true_y, self.pred_y), "float"), name="accuracy")

In [6]:
# 定义性能指标函数


def mean(item):
    return sum(item) / len(item)


def get_chunk_type(index, index_to_label):
    """
    对实体的标签进行分割，返回实体的位置和实体的名称
    """
    label_name = index_to_label[index]
    label_class, label_type = label_name.split("-")

    return label_name, label_type


def get_chunk(sequence, label_to_index):
    """
    给定一个标注序列，将实体和位置组合起来，放置在一个列表中
    """
    unentry = [label_to_index["O"]]
    index_to_label = {index: label for label, index in label_to_index.items()}
    chunks = []
    chunk_type, chunk_start = None, None
    for index, label in enumerate(sequence):
        if label in unentry:
            # 如果非实体词
            if chunk_type is None:
                # 若chunk_type为None，表明上一个词是非实体，继续跳过
                continue
            else:
                # 若chunkType非None，则上面的是一个实体，而当前非实体，则将上一个实体chunk加入到chunks中
                # 主要为序列中的这种情况，O,B-PER,I-PER,O 这也是最常见的情况
                chunk = (chunk_type, chunk_start, index-1)
                chunks.append(chunk)
                chunk_type, chunk_start = None, None

        if label not in unentry:
            # 如果是实体词，在这里的label是索引表示的label
            label_chunk_class, label_chunk_type = get_chunk_type(label, index_to_label)
            if chunk_type is None:
                # 若当前chunk_type为None，则表明上一个词是非实体词
                chunk_type, chunk_start = label_chunk_type, index
            elif label_chunk_type == chunk_type:
                # 若实体类型和上一个相同，则做如下判断
                if index == (len(sequence) - 1):
                    # 若当前词是序列中的最后一个词，则直接返回chunk
                    chunk = (chunk_type, chunk_start, index)
                    chunks.append(chunk)
                else:
                    # 若当前非最后一个词，则跳过
                    continue
            elif label_chunk_type != chunk_type:
                # 若当前词和上一个词类型不同，则将上一个实体chunk加入到chunks中，接着继续下一个chunk
                # 主要体现在两个实体相连的序列中，如B-PER,I-PER,B-LOC,I-LOC
                chunk = (chunk_type, chunk_start, index-1)
                chunks.append(chunk)
                chunk_type, chunk_start = label_chunk_type, index

    return chunks


def gen_metrics(true_y, pred_y, label_to_index):
    """
    生成f1值，recall, precision
    precision = 识别的正确实体数/识别出的实体数
    recall = 识别的正确实体数/样本的实体数
    """
    correct_preds = 0  # 识别出的正确实体数
    all_preds = 0  # 识别出的实体数
    all_trues = 0  # 样本的真实实体数

    true_chunks = get_chunk(true_y.tolist(), label_to_index)
    pred_chunks = get_chunk(pred_y.tolist(), label_to_index)
    correct_preds += len(set(true_chunks) & set(pred_chunks))
    all_preds += len(pred_chunks)
    all_trues += len(true_chunks)

    precision = correct_preds / all_preds if correct_preds > 0 else 0
    recall = correct_preds / all_trues if correct_preds > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if correct_preds > 0 else 0

    return round(f1, 4), round(precision, 4), round(recall, 4)

In [ ]:
# 训练模型

# 生成训练集和验证集
train_datas = dataGen.train_datas
train_labels = dataGen.train_labels
eval_datas = dataGen.eval_datas
eval_labels = dataGen.eval_labels

vocab_len = dataGen.vocab_len
label_to_index = dataGen.label_to_index

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = BiLSTMCrf(config, vocab_len, True)
        
        global_step = tf.Variable(0, name="global_step", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learning_rate)
        # 计算梯度,得到梯度和变量
        grads_and_vars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # 用summary绘制tensorBoard
#         grad_summaries = []
#         for g, v in grads_and_vars:
#             if g is not None:
#                 tf.summary.histogram("{}/grad/hist".format(v.name), g)
#                 tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
#         out_dir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
#         print("Writing to {}\n".format(out_dir))
        
#         loss_summary = tf.summary.scalar("loss", lstm.loss)
#         summary_op = tf.summary.merge_all()
        
#         train_summary_dir = os.path.join(out_dir, "train")
#         train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        
#         eval_summary_dir = os.path.join(out_dir, "eval")
#         eval_summary_writer = tf.summary.FileWriter(eval_summary_dir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        sess.run(tf.global_variables_initializer())

        def train_step(batch_x, batch_y, sequence_len, max_len):
            """
            训练函数
            """   
            feed_dict = {
                lstm.input_x: batch_x,
                lstm.input_y: batch_y,
                lstm.max_len: max_len,
                lstm.sequence_len: sequence_len,
                lstm.dropout_keep_prob: config.model.dropout_keep_prob
            }
        
            _, step, loss, acc, true_y, pred_y = sess.run(
                [train_op, global_step, lstm.loss, lstm.accuracy, lstm.true_y, lstm.pred_y], feed_dict)
            time_str = datetime.datetime.now().isoformat()

            f1, precision, recall = gen_metrics(true_y, pred_y, label_to_index)
            
            print("{}, step: {}, loss: {}, acc: {}, f1: {}, precision: {}, recall: {}".format(time_str, step, loss, 
                                                                                              acc, f1, precision, recall))
#             train_summary_writer.add_summary(summary, step)

        def dev_step(batch_x, batch_y, sequence_len, max_len):
            """
            验证函数
            """
            feed_dict = {
                lstm.input_x: batch_x,
                lstm.input_y: batch_y,
                lstm.max_len: max_len,
                lstm.sequence_len: sequence_len,
                lstm.dropout_keep_prob: 1.0
            }
            step, loss, acc, true_y, pred_y = sess.run(
                [global_step, lstm.loss, lstm.accuracy, lstm.true_y, lstm.pred_y], feed_dict)
            
            f1, precision, recall = gen_metrics(true_y, pred_y, label_to_index)
            
#             evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, f1, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batch_train in dataGen.next_batch(train_datas, train_labels):
                train_step(batch_train["input_x"], batch_train["input_y"], batch_train["sequence_len"], batch_train["max_len"])

                current_step = tf.train.global_step(sess, global_step) 
                if current_step % config.training.evaluate_every == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accuracys = []
                    f1s = []
                    precisions = []
                    recalls = []
                    for batch_eval in dataGen.next_batch(eval_datas, eval_labels):
                        loss, accuracy, f1, precision, recall = dev_step(batch_eval["input_x"], batch_eval["input_y"], 
                                                                         batch_eval["sequence_len"], batch_eval["max_len"])
                        losses.append(loss)
                        accuracys.append(accuracy)
                        f1s.append(f1)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc:{}, f1: {}, precision: {}, recall: {}".format(time_str, current_step, mean(losses), 
                                                                                                     mean(accuracys), mean(f1s), 
                                                                                                     mean(precisions), mean(recalls)))
                    print("\n")
                    
#                 if current_step % config.training.checkpoint_every == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=current_step)
#                     print("Saved model checkpoint to {}\n".format(path))
                    

start training model
2019-01-24T10:11:04.467656, step: 1, loss: 83.59038543701172, acc: 0.12784899771213531, f1: 0.002, precision: 0.0011, recall: 0.0109
2019-01-24T10:11:04.795430, step: 2, loss: 75.69819641113281, acc: 0.3560681939125061, f1: 0.0039, precision: 0.0021, recall: 0.0235
2019-01-24T10:11:05.456501, step: 3, loss: 73.52793884277344, acc: 0.5631009340286255, f1: 0.0094, precision: 0.0054, recall: 0.037
2019-01-24T10:11:05.942013, step: 4, loss: 55.293800354003906, acc: 0.7597693800926208, f1: 0.0093, precision: 0.006, recall: 0.0211
2019-01-24T10:11:06.562464, step: 5, loss: 38.790428161621094, acc: 0.8548153638839722, f1: 0, precision: 0, recall: 0
2019-01-24T10:11:06.963134, step: 6, loss: 34.26868438720703, acc: 0.8697394728660583, f1: 0, precision: 0, recall: 0
2019-01-24T10:11:08.452968, step: 7, loss: 38.830909729003906, acc: 0.8178199529647827, f1: 0, precision: 0, recall: 0
2019-01-24T10:11:08.801356, step: 8, loss: 27.227787017822266, acc: 0.8896216750144958, f1: 

2019-01-24T10:11:38.560686, step: 68, loss: 20.347476959228516, acc: 0.8904503583908081, f1: 0.0255, precision: 0.05, recall: 0.0171
2019-01-24T10:11:38.834244, step: 69, loss: 16.353452682495117, acc: 0.8919222354888916, f1: 0.0357, precision: 0.0833, recall: 0.0227
2019-01-24T10:11:39.354665, step: 70, loss: 19.312713623046875, acc: 0.9001782536506653, f1: 0, precision: 0, recall: 0
2019-01-24T10:11:39.740496, step: 71, loss: 22.19683074951172, acc: 0.870347797870636, f1: 0.0127, precision: 0.0227, recall: 0.0088
2019-01-24T10:11:40.074765, step: 72, loss: 17.679569244384766, acc: 0.8913357257843018, f1: 0.0303, precision: 0.0588, recall: 0.0204
2019-01-24T10:11:40.469641, step: 73, loss: 19.829833984375, acc: 0.8883216381072998, f1: 0.0146, precision: 0.0303, recall: 0.0096
2019-01-24T10:11:40.795292, step: 74, loss: 19.325138092041016, acc: 0.8873653411865234, f1: 0, precision: 0, recall: 0
2019-01-24T10:11:41.331859, step: 75, loss: 27.236433029174805, acc: 0.8499407172203064, f1:

2019-01-24T10:12:24.299726, step: 129, loss: 16.442401885986328, acc: 0.8755655884742737, f1: 0.1279, precision: 0.1443, recall: 0.1148
2019-01-24T10:12:24.747681, step: 130, loss: 13.915472030639648, acc: 0.913913905620575, f1: 0.1637, precision: 0.1867, recall: 0.1458
2019-01-24T10:12:25.340785, step: 131, loss: 15.023956298828125, acc: 0.9092387557029724, f1: 0.1333, precision: 0.1477, recall: 0.1215
2019-01-24T10:12:25.633431, step: 132, loss: 13.909459114074707, acc: 0.9068493247032166, f1: 0.1828, precision: 0.2125, recall: 0.1604
2019-01-24T10:12:26.214140, step: 133, loss: 17.706741333007812, acc: 0.9045284390449524, f1: 0.141, precision: 0.1553, recall: 0.129
2019-01-24T10:12:26.679242, step: 134, loss: 17.092544555664062, acc: 0.8928241729736328, f1: 0.1475, precision: 0.1702, recall: 0.1301
2019-01-24T10:12:27.033923, step: 135, loss: 17.265050888061523, acc: 0.8984974026679993, f1: 0.125, precision: 0.1548, recall: 0.1048
2019-01-24T10:12:27.698008, step: 136, loss: 10.5979

2019-01-24T10:12:52.926563, step: 190, loss: 9.346099853515625, acc: 0.9285714030265808, f1: 0.2333, precision: 0.2234, recall: 0.2442
2019-01-24T10:12:53.470445, step: 191, loss: 9.912862777709961, acc: 0.9354838728904724, f1: 0.2532, precision: 0.2817, recall: 0.2299
2019-01-24T10:12:53.770770, step: 192, loss: 9.200638771057129, acc: 0.936796247959137, f1: 0.2375, precision: 0.2436, recall: 0.2317
2019-01-24T10:12:54.091702, step: 193, loss: 13.177605628967285, acc: 0.9035874605178833, f1: 0.1925, precision: 0.1901, recall: 0.1949
2019-01-24T10:12:54.531587, step: 194, loss: 11.692618370056152, acc: 0.9120956659317017, f1: 0.2515, precision: 0.2763, recall: 0.2308
2019-01-24T10:12:54.907279, step: 195, loss: 11.049695014953613, acc: 0.9242724776268005, f1: 0.2353, precision: 0.2623, recall: 0.2133
2019-01-24T10:12:55.163429, step: 196, loss: 8.704599380493164, acc: 0.9427279829978943, f1: 0.2794, precision: 0.2923, recall: 0.2676
2019-01-24T10:12:55.712120, step: 197, loss: 13.25269

2019-01-24T10:14:06.708686, step: 250, loss: 8.756786346435547, acc: 0.942756175994873, f1: 0.3611, precision: 0.4333, recall: 0.3095
2019-01-24T10:14:07.420690, step: 251, loss: 8.113692283630371, acc: 0.9343236684799194, f1: 0.2857, precision: 0.2874, recall: 0.2841
2019-01-24T10:14:08.225691, step: 252, loss: 10.389699935913086, acc: 0.932695209980011, f1: 0.2826, precision: 0.2989, recall: 0.268
2019-01-24T10:14:09.040766, step: 253, loss: 9.365396499633789, acc: 0.9357551336288452, f1: 0.3593, precision: 0.3704, recall: 0.3488
2019-01-24T10:14:09.664371, step: 254, loss: 10.805408477783203, acc: 0.93019038438797, f1: 0.274, precision: 0.2804, recall: 0.2679
2019-01-24T10:14:10.250714, step: 255, loss: 8.68299388885498, acc: 0.9375629425048828, f1: 0.4356, precision: 0.419, recall: 0.4536
2019-01-24T10:14:10.733681, step: 256, loss: 7.572927474975586, acc: 0.9507065415382385, f1: 0.3478, precision: 0.3111, recall: 0.3944
2019-01-24T10:14:11.614458, step: 257, loss: 9.24523162841796

...

2019-01-24T10:38:26.860964, step: 1635, loss: 2.087620258331299, acc: 0.9749364256858826, f1: 0.656, precision: 0.6613, recall: 0.6508
2019-01-24T10:38:27.343089, step: 1636, loss: 2.081826686859131, acc: 0.9829989671707153, f1: 0.8317, precision: 0.866, recall: 0.8
2019-01-24T10:38:27.885329, step: 1637, loss: 2.042146682739258, acc: 0.984375, f1: 0.8, precision: 0.8046, recall: 0.7955
2019-01-24T10:38:28.770813, step: 1638, loss: 2.2281126976013184, acc: 0.9786259531974792, f1: 0.6861, precision: 0.7344, recall: 0.6438
2019-01-24T10:38:29.273219, step: 1639, loss: 1.9346365928649902, acc: 0.9824561476707458, f1: 0.8597, precision: 0.8407, recall: 0.8796
2019-01-24T10:38:29.764026, step: 1640, loss: 1.4889405965805054, acc: 0.9891945123672485, f1: 0.8217, precision: 0.8413, recall: 0.803
2019-01-24T10:38:30.746900, step: 1641, loss: 1.6003960371017456, acc: 0.9864524006843567, f1: 0.8402, precision: 0.8765, recall: 0.8068
2019-01-24T10:38:31.325583, step: 1642, loss: 2.023778438568115

2019-01-24T10:39:15.869422, step: 1696, loss: 2.2335093021392822, acc: 0.9803125262260437, f1: 0.8098, precision: 0.8218, recall: 0.7981
2019-01-24T10:39:16.489237, step: 1697, loss: 1.7879674434661865, acc: 0.9828340411186218, f1: 0.8024, precision: 0.8272, recall: 0.7791
2019-01-24T10:39:17.189853, step: 1698, loss: 2.8321709632873535, acc: 0.9758298397064209, f1: 0.7222, precision: 0.7647, recall: 0.6842
2019-01-24T10:39:18.413231, step: 1699, loss: 2.791677713394165, acc: 0.9818748235702515, f1: 0.7983, precision: 0.8304, recall: 0.7686
2019-01-24T10:39:19.016261, step: 1700, loss: 2.6914093494415283, acc: 0.9627659320831299, f1: 0.77, precision: 0.781, recall: 0.7593

Evaluation:
2019-01-24T10:39:43.858752, step: 1700, loss: 2.83412793940968, acc:0.9724232041173511, f1: 0.7687749999999999, precision: 0.790661111111111, recall: 0.7486638888888889


2019-01-24T10:39:44.374234, step: 1701, loss: 2.918635368347168, acc: 0.964091420173645, f1: 0.7297, precision: 0.7431, recall: 0.7168


2019-01-24T10:40:29.635280, step: 1755, loss: 3.211209774017334, acc: 0.9737258553504944, f1: 0.7435, precision: 0.7717, recall: 0.7172
2019-01-24T10:40:30.514790, step: 1756, loss: 2.9342668056488037, acc: 0.9726890921592712, f1: 0.757, precision: 0.757, recall: 0.757
2019-01-24T10:40:31.423187, step: 1757, loss: 1.9588146209716797, acc: 0.979126513004303, f1: 0.8085, precision: 0.8261, recall: 0.7917
2019-01-24T10:40:32.155821, step: 1758, loss: 1.4291329383850098, acc: 0.9886363744735718, f1: 0.8378, precision: 0.8378, recall: 0.8378
2019-01-24T10:40:32.924451, step: 1759, loss: 2.5608949661254883, acc: 0.9741322994232178, f1: 0.79, precision: 0.79, recall: 0.79
2019-01-24T10:40:33.805032, step: 1760, loss: 2.1234183311462402, acc: 0.9793035387992859, f1: 0.7707, precision: 0.7745, recall: 0.767
2019-01-24T10:40:34.505070, step: 1761, loss: 2.376509666442871, acc: 0.9731733202934265, f1: 0.7273, precision: 0.7423, recall: 0.7129
2019-01-24T10:40:35.024314, step: 1762, loss: 1.989679

2019-01-24T10:41:37.120372, step: 1814, loss: 2.458761692047119, acc: 0.9763978719711304, f1: 0.784, precision: 0.8033, recall: 0.7656
2019-01-24T10:41:37.653462, step: 1815, loss: 1.6837632656097412, acc: 0.984455943107605, f1: 0.8191, precision: 0.8105, recall: 0.828
2019-01-24T10:41:38.467457, step: 1816, loss: 2.260334014892578, acc: 0.9768258333206177, f1: 0.7407, precision: 0.7609, recall: 0.7216
2019-01-24T10:41:39.479182, step: 1817, loss: 1.6416460275650024, acc: 0.9810725450515747, f1: 0.8621, precision: 0.9036, recall: 0.8242
2019-01-24T10:41:40.219437, step: 1818, loss: 1.5171003341674805, acc: 0.9886121153831482, f1: 0.8673, precision: 0.8854, recall: 0.85
2019-01-24T10:41:40.736610, step: 1819, loss: 2.2456374168395996, acc: 0.97620689868927, f1: 0.7386, precision: 0.7558, recall: 0.7222
2019-01-24T10:41:41.658902, step: 1820, loss: 1.728732705116272, acc: 0.9798431992530823, f1: 0.7784, precision: 0.7831, recall: 0.7738
2019-01-24T10:41:44.175667, step: 1821, loss: 3.371

2019-01-24T10:42:29.911053, step: 1875, loss: 2.5376791954040527, acc: 0.9737171530723572, f1: 0.7811, precision: 0.7982, recall: 0.7647
2019-01-24T10:42:30.463679, step: 1876, loss: 2.370288372039795, acc: 0.9716412425041199, f1: 0.7729, precision: 0.8, recall: 0.7477
2019-01-24T10:42:31.013829, step: 1877, loss: 2.310115337371826, acc: 0.98175048828125, f1: 0.7625, precision: 0.7922, recall: 0.7349
2019-01-24T10:42:31.760605, step: 1878, loss: 1.7339906692504883, acc: 0.9837962985038757, f1: 0.8458, precision: 0.8649, recall: 0.8276
2019-01-24T10:42:34.053014, step: 1879, loss: 2.875293016433716, acc: 0.9781083464622498, f1: 0.8114, precision: 0.8261, recall: 0.7972
2019-01-24T10:42:34.924293, step: 1880, loss: 1.8289082050323486, acc: 0.9811260104179382, f1: 0.7959, precision: 0.8211, recall: 0.7723
2019-01-24T10:42:35.946684, step: 1881, loss: 2.8372416496276855, acc: 0.9719222187995911, f1: 0.7373, precision: 0.7767, recall: 0.7018
2019-01-24T10:42:36.707813, step: 1882, loss: 2.0